In [49]:
from random import randint


def null_score(game, player):
    """This heuristic presumes no knowledge for non-terminal states, and
    returns the same uninformative value for all other states.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).

    Returns
    ----------
    float
        The heuristic value of the current game state.
    """

    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    return 0.


def open_move_score(game, player):
    """The basic evaluation function described in lecture that outputs a score
    equal to the number of moves open for your computer player on the board.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).

    Returns
    ----------
    float
        The heuristic value of the current game state
    """
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    return float(len(game.get_legal_moves(player)))


def improved_score(game, player):
    """The "Improved" evaluation function discussed in lecture that outputs a
    score equal to the difference in the number of moves available to the
    two players.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).

    Returns
    ----------
    float
        The heuristic value of the current game state
    """
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    own_moves = len(game.get_legal_moves(player))
    opp_moves = len(game.get_legal_moves(game.get_opponent(player)))
    return float(own_moves - opp_moves)


def center_score(game, player):
    """Outputs a score equal to square of the distance from the center of the
    board to the position of the player.

    This heuristic is only used by the autograder for testing.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).

    Returns
    ----------
    float
        The heuristic value of the current game state
    """
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    w, h = game.width / 2., game.height / 2.
    y, x = game.get_player_location(player)
    return float((h - y)**2 + (w - x)**2)


class RandomPlayer():
    """Player that chooses a move randomly."""

    def get_move(self, game, time_left):
        """Randomly select a move from the available legal moves.

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        ----------
        (int, int)
            A randomly selected legal move; may return (-1, -1) if there are
            no available legal moves.
        """
        legal_moves = game.get_legal_moves()
        if not legal_moves:
            return (-1, -1)
        return legal_moves[randint(0, len(legal_moves) - 1)]


class GreedyPlayer():
    """Player that chooses next move to maximize heuristic score. This is
    equivalent to a minimax search agent with a search depth of one.
    """

    def __init__(self, score_fn=open_move_score):
        self.score = score_fn

    def get_move(self, game, time_left):
        """Select the move from the available legal moves with the highest
        heuristic score.

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        ----------
        (int, int)
            The move in the legal moves list with the highest heuristic score
            for the current game state; may return (-1, -1) if there are no
            legal moves.
        """
        legal_moves = game.get_legal_moves()
        if not legal_moves:
            return (-1, -1)
        _, move = max([(self.score(game.forecast_move(m), self), m) for m in legal_moves])
        return move


class HumanPlayer():
    """Player that chooses a move according to user's input."""

    def get_move(self, game, time_left):
        """
        Select a move from the available legal moves based on user input at the
        terminal.

        **********************************************************************
        NOTE: If testing with this player, remember to disable move timeout in
              the call to `Board.play()`.
        **********************************************************************

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        ----------
        (int, int)
            The move in the legal moves list selected by the user through the
            terminal prompt; automatically return (-1, -1) if there are no
            legal moves
        """
        legal_moves = game.get_legal_moves()
        if not legal_moves:
            return (-1, -1)

        print(game.to_string()) #display the board for the human player
        print(('\t'.join(['[%d] %s' % (i, str(move)) for i, move in enumerate(legal_moves)])))

        valid_choice = False
        while not valid_choice:
            try:
                index = int(input('Select move index:'))
                valid_choice = 0 <= index < len(legal_moves)

                if not valid_choice:
                    print('Illegal move! Try again.')

            except ValueError:
                print('Invalid index! Try again.')

        return legal_moves[index]

class Minimax():
    
    def minimax(self, game, depth):
        """Implement depth-limited minimax search algorithm as described in
        the lectures.

        This should be a modified version of MINIMAX-DECISION in the AIMA text.
        https://github.com/aimacode/aima-pseudocode/blob/master/md/Minimax-Decision.md

        **********************************************************************
            You MAY add additional methods to this class, or define helper
                 functions to implement the required functionality.
        **********************************************************************

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        Returns
        -------
        (int, int)
            The board coordinates of the best move found in the current search;
            (-1, -1) if there are no legal moves

        Notes
        -----
            (1) You MUST use the `self.score()` method for board evaluation
                to pass the project tests; you cannot call any other evaluation
                function directly.

            (2) If you use any helper functions (e.g., as shown in the AIMA
                pseudocode) then you must copy the timer check into the top of
                each helper function or else your agent will timeout during
                testing.
        """
        #if self.time_left() < self.TIMER_THRESHOLD:
         #   raise SearchTimeout()

        # TODO: finish this function!
        # Starting the value to control the depth of the search
        i_depth=0
        # Initializing the lis to store the utilities values
        utility=[]
        # First I get the legal moves for the actual state of the board
        legal_moves=game.get_legal_moves()
        # I made an iteration to get the utility value for each move aplying minmax
        for idx in range(len(legal_moves)):
            #I get a new game apliying the first move
            new_game= game.forecast_move(legal_moves[idx])
            utility=utility.append(min_value(new_game,i_depth,depth))
        #Taking the maximun value from the utility
        max_utility=max(utility)
            
        return legal_moves[utility.index(max_utility)]
    
        #raise NotImplementedError

    def max_value(self,game, i_depth, depth):
        """ Implement a function to obtain the max value of a tree 
        
        
        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        i_depth : int
            i_depth represent the plie in wich we are searching
        
        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting
        

        Returns
        -------
        int        
            The utility value for the movement
        """
        utility=[]
        i_depth+=1
        
        if terminal_test(game) or (i_depth>depth):
            return self.score(game)
        
        legal_moves=game.get_legal_moves()
        
        for idx in range(len(legal_moves)):
            new_game=game.forecast_move(legal_moves[idx])
            utility=utility.append(min_value(new_game))
        
        return max(utility)
    
    def min_value(self,game,i_depth, depth):
        """ Implement a function to obtain the min value of a tree 
        
        
        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state
        
        i_depth : int
            i_depth represent the plie in wich we are searching

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        Returns
        -------
        int        
            The utility value for the movement
        """
        utility=[]
        i_depth+=1
        
        if terminal_test(game) or (i_depth>depth):
            return self.score(game)
        
        legal_moves=game.get_legal_moves()
        
        for idx in range(len(legal_moves)):
            new_game=game.forecast_move(legal_moves[idx])
            utility=utility.append(max_value(new_game))
        
        return min(utility)
        
    def terminal_test(self,game):
        """ Determine if the game is at the end for the player
        
        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state
        
        Returns
        -------
        boolean        
            if the legal_moves=0 then is the end of the game 
        """
            
        moves=game.get_legal_moves()
        
        if len(moves)==0:
            return True
        return False
    

In [50]:
from isolation import Board

In [51]:
player1 = RandomPlayer()
player2 = GreedyPlayer()
game = Board(player1, player2)

In [52]:
minimax=Minimax()

In [53]:
minimax.minimax(game,3)

TypeError: terminal_test() missing 1 required positional argument: 'game'

In [36]:
minimax=Minimax()

In [37]:
minimax.terminal_test(game)

False

In [8]:
len(legal_moves)

8

In [11]:
for idx in range(len(legal_moves)):
    print(idx)
    print(legal_moves[idx])

0
(4, 2)
1
(1, 1)
2
(3, 1)
3
(0, 2)
4
(1, 5)
5
(4, 4)
6
(0, 4)
7
(3, 5)


In [13]:
new_game = game.forecast_move(legal_moves[0])

In [15]:
print(new_game.to_string())

     0   1   2   3   4   5   6
0  |   |   |   |   |   | 2 |   | 
1  |   |   |   |   |   |   |   | 
2  |   |   |   | - |   |   |   | 
3  |   |   |   |   |   |   |   | 
4  |   |   | 1 |   |   |   |   | 
5  |   |   |   |   |   |   |   | 
6  |   |   |   |   |   |   |   | 



In [17]:
if len(legal_moves)==0:
    print("true")
print("false")

false


In [19]:
lista=[]

In [20]:
lista.append(1)
lista.append(2)
lista.append(3)
lista.append(4)
lista.append(5)
print(lista)

[1, 2, 3, 4, 5]


In [21]:
min(lista)

1

In [22]:
max(lista)

5

In [38]:
lista.index(3)

2

In [39]:
pepe=1
pepe+=1


In [40]:
pepe

2